<a href="https://colab.research.google.com/github/woojung02/SSAC_AI/blob/main/DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch                                   # PyTorch 기본 모듈
import torch.nn as nn                          # 신경망 레이어(nn.Module, Conv2d, BatchNorm2d, Linear 등)
import torch.optim as optim                     # 최적화 알고리즘 (SGD, Adam 등)
import torchvision
import torchvision.transforms as transforms    # 데이터 전처리 & 증강
from torch.utils.data import DataLoader       # 데이터 로더

# 1) 장치 설정 (GPU 사용 가능하면 GPU, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) CIFAR-10 전처리 및 증강 정의
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),      # 작은 패딩을 붙이고 랜덤 크롭 → 과적합 방지
    transforms.RandomHorizontalFlip(),         # 좌우 반전 → 데이터 다양성 증가
    transforms.ToTensor(),                     # [0,255]→[0,1] Tensor
    transforms.Normalize((0.4914,0.4822,0.4465),# 채널별 평균으로 정규화 → 학습 안정화
                         (0.2023,0.1994,0.2010)),# 채널별 표준편차로 스케일링
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])

# 3) 데이터셋 & 로더 준비
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform_test)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True,  num_workers=2)
testloader  = DataLoader(testset,  batch_size=100, shuffle=False, num_workers=2)

# 4) Dense Layer (Bottleneck) 정의
class _DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate, bn_size, drop_rate):
        super().__init__()
        # 1×1 Bottleneck conv (채널 압축)
        self.bn1   = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, bn_size*growth_rate,
                               kernel_size=1, stride=1, bias=False)
        # 3×3 conv (실제 특징 추출)
        self.bn2   = nn.BatchNorm2d(bn_size*growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(bn_size*growth_rate, growth_rate,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.drop_rate = drop_rate

    def forward(self, x):
        # x: [B, in_channels, H, W]
        out = self.conv1(self.relu1(self.bn1(x)))
        out = self.conv2(self.relu2(self.bn2(out)))
        if self.drop_rate > 0:
            out = nn.functional.dropout(out, p=self.drop_rate, training=self.training)
        # Dense connection: 입력에 새로 만든 특징 맵을 채널 차원으로 이어붙임
        return torch.cat([x, out], 1)

# 5) Dense Block 정의 (여러 개의 DenseLayer를 쌓음)
class _DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate, bn_size, drop_rate):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(_DenseLayer(
                in_channels + i*growth_rate,  # 매 레이어마다 채널이 growth_rate만큼 늘어남
                growth_rate, bn_size, drop_rate
            ))
        self.denseblock = nn.Sequential(*layers)

    def forward(self, x):
        return self.denseblock(x)

# 6) Transition Layer 정의 (채널/공간 크기 축소)
class _Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.bn    = nn.BatchNorm2d(in_channels)
        self.relu  = nn.ReLU(inplace=True)
        self.conv  = nn.Conv2d(in_channels, out_channels,
                               kernel_size=1, stride=1, bias=False)
        self.pool  = nn.AvgPool2d(kernel_size=2, stride=2)  # 절반 크기로 다운샘플링

    def forward(self, x):
        return self.pool(self.conv(self.relu(self.bn(x))))

# 7) 전체 DenseNet 정의
class DenseNet(nn.Module):
    def __init__(self, growth_rate=12, block_layers=(6,6,6), # CIFAR용 작은 DenseNet-BC
                 num_init_features=2*12, bn_size=4, drop_rate=0, num_classes=10):
        super().__init__()
        # (1) 초기 컨브: 3채널→num_init_features, 크기 유지(padding=1)
        self.features = nn.Sequential(
            nn.Conv2d(3, num_init_features, kernel_size=3,
                      stride=1, padding=1, bias=False),
        )
        num_features = num_init_features

        # (2) DenseBlock + Transition 쌓기
        for i, num_layers in enumerate(block_layers):
            # DenseBlock
            block = _DenseBlock(num_layers, num_features, growth_rate, bn_size, drop_rate)
            self.features.add_module(f"denseblock{i+1}", block)
            num_features += num_layers * growth_rate

            # 마지막 블록 뒤에는 Transition 추가하고 싶지 않음
            if i != len(block_layers)-1:
                trans = _Transition(num_features, num_features // 2)
                self.features.add_module(f"transition{i+1}", trans)
                num_features = num_features // 2

        # (3) 최종 BatchNorm+ReLU
        self.features.add_module("norm_final", nn.BatchNorm2d(num_features))
        self.features.add_module("relu_final", nn.ReLU(inplace=True))

        # (4) 분류기: 전역 평균 풀링 + FC
        self.classifier = nn.Linear(num_features, num_classes)

        # 가중치 초기화
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)  # He 초기화
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)     # scale=1
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # 특징 추출
        features = self.features(x)          # [B, C, H, W]
        # 전역 평균 풀링 (H×W → 1×1)
        out = nn.functional.adaptive_avg_pool2d(features, (1,1))
        out = torch.flatten(out, 1)          # [B, C]
        # 분류기
        return self.classifier(out)

# 8) 모델 생성 & 옵티마이저/손실함수/스케줄러
model = DenseNet().to(device)               # CIFAR-10용 기본 설정
criterion = nn.CrossEntropyLoss()           # 분류용 손실
optimizer = optim.SGD(model.parameters(),
                      lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[50,100,150], gamma=0.1)

# 9) 학습 루프 & 평가 함수
def train(epoch):
    model.train()
    total_loss = 0
    total_correct = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss    += loss.item()*images.size(0)
        total_correct += (outputs.argmax(1)==labels).sum().item()

    print(f"[Train] Epoch {epoch}: "
          f"Loss={total_loss/len(trainset):.4f}, "
          f"Acc={100*total_correct/len(trainset):.2f}%")

def test(epoch):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss    += loss.item()*images.size(0)
            total_correct += (outputs.argmax(1)==labels).sum().item()

    print(f"[Test ] Epoch {epoch}: "
          f"Loss={total_loss/len(testset):.4f}, "
          f"Acc={100*total_correct/len(testset):.2f}%")

# 10) 전체 학습 실행
num_epochs = 100
for epoch in range(1, num_epochs+1):
    train(epoch)
    test(epoch)
    scheduler.step()


100%|██████████| 170M/170M [00:03<00:00, 45.0MB/s]


[Train] Epoch 1: Loss=1.4637, Acc=45.77%
[Test ] Epoch 1: Loss=1.3252, Acc=55.16%
[Train] Epoch 2: Loss=0.9811, Acc=65.14%
[Test ] Epoch 2: Loss=1.0302, Acc=64.70%
[Train] Epoch 3: Loss=0.8015, Acc=71.81%
[Test ] Epoch 3: Loss=0.7716, Acc=73.64%
[Train] Epoch 4: Loss=0.6889, Acc=76.10%
[Test ] Epoch 4: Loss=0.7338, Acc=75.48%
[Train] Epoch 5: Loss=0.6140, Acc=78.73%
[Test ] Epoch 5: Loss=0.7166, Acc=75.89%
[Train] Epoch 6: Loss=0.5710, Acc=80.25%
[Test ] Epoch 6: Loss=0.7046, Acc=76.33%
[Train] Epoch 7: Loss=0.5229, Acc=81.93%
[Test ] Epoch 7: Loss=0.6138, Acc=79.66%
[Train] Epoch 8: Loss=0.4962, Acc=83.03%
[Test ] Epoch 8: Loss=0.6807, Acc=77.94%
[Train] Epoch 9: Loss=0.4726, Acc=83.59%
[Test ] Epoch 9: Loss=0.6252, Acc=79.21%
[Train] Epoch 10: Loss=0.4483, Acc=84.46%
[Test ] Epoch 10: Loss=0.6086, Acc=79.88%
[Train] Epoch 11: Loss=0.4317, Acc=85.00%
[Test ] Epoch 11: Loss=0.5693, Acc=81.71%
[Train] Epoch 12: Loss=0.4211, Acc=85.34%
[Test ] Epoch 12: Loss=0.5632, Acc=81.53%
[Train] Ep

In [2]:
import torch                                      # PyTorch 기본 모듈
import torch.nn as nn                             # 신경망 레이어 & 손실함수
import torch.optim as optim                        # 최적화 알고리즘
import torchvision
from torchvision import datasets, transforms      # 데이터셋 & 전처리
from torch.utils.data import DataLoader            # 배치 단위 로더
from torchvision.models import densenet121         # TorchVision 제공 DenseNet-121

# 1) 장치 설정: GPU 가능 시 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) CIFAR-10 전처리 정의 (학습 시 증강 포함)
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),    # 패딩 후 랜덤 크롭: 과적합 방지
    transforms.RandomHorizontalFlip(),       # 좌우 뒤집기: 데이터 다양성
    transforms.ToTensor(),                   # [0,255]→[0,1] 텐서 변환
    transforms.Normalize((0.4914,0.4822,0.4465),  # 채널별 평균으로 정규화
                         (0.2023,0.1994,0.2010))  # 채널별 표준편차로 스케일링
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),
                         (0.2023,0.1994,0.2010)),
])

# 3) 데이터셋 & 데이터로더 생성
trainset    = datasets.CIFAR10(root='./data', train=True,  download=True, transform=transform_train)
testset     = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True,  num_workers=2)
testloader  = DataLoader(testset,  batch_size=100, shuffle=False, num_workers=2)

# 4) 사전 정의된 DenseNet-121 모델 불러와서 CIFAR-10용으로 수정
model = densenet121(pretrained=False, num_classes=10).to(device)

# 5) 손실함수 & 옵티마이저 & 스케줄러 설정
criterion = nn.CrossEntropyLoss()                  # 다중 클래스 분류 손실
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,100,150], gamma=0.1)
# milestones마다 lr 0.1배 감소

# 6) 학습 함수
def train(epoch):
    model.train()                                 # 드롭아웃/배치정규화 활성화
    running_loss, correct, total = 0.0, 0, 0
    for imgs, labels in trainloader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()                    # 기울기 초기화
        outputs = model(imgs)                    # 순전파
        loss = criterion(outputs, labels)        # 손실 계산
        loss.backward()                          # 역전파
        optimizer.step()                         # 파라미터 업데이트

        running_loss += loss.item()*labels.size(0)
        preds = outputs.argmax(dim=1)
        correct += preds.eq(labels).sum().item()
        total   += labels.size(0)
    print(f"[Train] Epoch {epoch}  Loss: {running_loss/total:.4f}  Acc: {100*correct/total:.2f}%")

# 7) 평가 함수
def test(epoch):
    model.eval()                                 # 평가 모드: 드롭아웃·배치정규화 비활성화
    loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in testloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss += criterion(outputs, labels).item()*labels.size(0)
            preds = outputs.argmax(dim=1)
            correct += preds.eq(labels).sum().item()
            total   += labels.size(0)
    print(f"[Test ] Epoch {epoch}  Loss: {loss/total:.4f}  Acc: {100*correct/total:.2f}%")

# 8) 전체 학습 루프
num_epochs = 20
for epoch in range(1, num_epochs+1):
    train(epoch)               # 학습 단계
    test(epoch)                # 테스트 단계
    scheduler.step()           # 학습률 스케줄러 업데이트


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[Train] Epoch 1  Loss: 2.1026  Acc: 31.15%
[Test ] Epoch 1  Loss: 1.7941  Acc: 35.32%
[Train] Epoch 2  Loss: 1.5169  Acc: 44.39%
[Test ] Epoch 2  Loss: 1.4341  Acc: 48.29%
[Train] Epoch 3  Loss: 1.3351  Acc: 51.58%
[Test ] Epoch 3  Loss: 1.2946  Acc: 53.58%
[Train] Epoch 4  Loss: 1.2052  Acc: 56.88%
[Test ] Epoch 4  Loss: 1.1116  Acc: 60.63%
[Train] Epoch 5  Loss: 1.0668  Acc: 61.75%
[Test ] Epoch 5  Loss: 0.9663  Acc: 65.59%
[Train] Epoch 6  Loss: 0.9932  Acc: 64.84%
[Test ] Epoch 6  Loss: 0.9881  Acc: 65.19%
[Train] Epoch 7  Loss: 0.9601  Acc: 65.71%
[Test ] Epoch 7  Loss: 0.9266  Acc: 67.86%
[Train] Epoch 8  Loss: 0.8939  Acc: 68.34%
[Test ] Epoch 8  Loss: 0.8519  Acc: 69.83%
[Train] Epoch 9  Loss: 0.8303  Acc: 70.66%
[Test ] Epoch 9  Loss: 0.8273  Acc: 71.24%
[Train] Epoch 10  Loss: 1.0293  Acc: 63.53%
[Test ] Epoch 10  Loss: 1.2369  Acc: 55.75%
[Train] Epoch 11  Loss: 1.0712  Acc: 61.99%
[Test ] Epoch 11  Loss: 0.9456  Acc: 66.76%
[Train] Epoch 12  Loss: 0.8792  Acc: 69.04%
[Test 